# To do:
- test similarity scikit image (Niko)
- add scope to antspy registration (Aicha) DONE!
   Also, moved the concatentation and sim_check of the registered files to the Antspy func
- change channel 2 shifted assignment to shift 'warpmoveout' (Aicha) DONE!

##Testing:
- stage 2: T4, T5
    - sessions: 220219, 220221
- stage 1: T4, T5 
    - sessions: 211011, 220124

Define your variables

In [1]:
ants_method = ['SyNRA']
path_to_data = '/home/mpg08/aicha.hajiali/TLI_project/TLI_data/2022/T4/220209/'
save_path = '/home/mpg08/aicha.hajiali/TLI_project/TLI_data/preprocessed/2022/T4/220209/SyNRA_old5/'
group_by = '209_brain2.2_LP40_P36'
ch_names = ['GFP', 'red']
save_name = '220209_brain2.2_LP40_P36'
csv_name = '_220209_SyNRA_antspy.csv'

Load packages and define functions

In [2]:
import ants
from skimage import io, filters
from tqdm import tqdm
import numpy as np
import os
from timeit import default_timer as timer
import tifffile as tif
import cv2 as cv
from sklearn import metrics
import csv

In [3]:
def get_file_names(path, group_by='', order=True, nested_files=False, criteria='tif'):
    """
    returns a list of all files' names in the given directory and its sub-folders
    the list can be filtered based on the 'group_by' str provided
    the files_list is sorted in reverse if the order is set to True. 
    The first element of the list is used later as ref
    """
    if os.path.isfile(path):
        file_list = [path]
    else:
        file_list = []
        if nested_files == False:
            file_list = [os.path.join(path, f) for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
        else:
            for path, subdirs, files in os.walk(path):
                for name in files:
                    file_list.append(os.path.join(path, name))
        file_list = [file for file in file_list if group_by in file]
        file_list = [file for file in file_list if criteria in file]
        file_list.sort(reverse=order)    
    return file_list

In [4]:
def split_convert(image, ch_names):
    """deinterleave the image into dictionary of two channels"""
    # for i in tqdm(range(1), desc = 'split_convert'):
    start_time = timer()
    image_ch = {}
    for ind, ch in enumerate(ch_names):
        image_ch[ch] = image[ind::len(ch_names)]
    # if len(ch_names) > 1:
    #     image_ch[ch_names[-1]] = filters.median(image_ch[ch_names[-1]])
    print('split_convert runtime', timer()-start_time)
    return image_ch

In [5]:
def files_to_4D(files_list, ch_names=[''], 
                save=True, save_path='', save_file='', 
                xy_pixel=1, z_pixel=1, ddtype='uint16'):
    """
    read files_list, load the individual 3D_img tifffiles, 
    and convert them into a dict of 4D-arrays of the identified ch
    has the option of saving is as 8uint image
    """
    start_time = timer()
    image_4D = {ch:[] for ch in ch_names}
    files_list.sort()
    for file in tqdm(files_list, desc = 'compiling_files'):
        image = tif.imread(file)
        image = split_convert(image, ch_names=ch_names)
        for ch in ch_names:
            image_4D[ch].append(image[ch])
    z_dim = min([len(img) for img in image_4D[ch_names[0]]])
    print(image_4D.keys(), type(image_4D[ch_names[-1]]), len(image_4D[ch_names[-1]]))
    for ch in ch_names:
        print('compiling the', ch, 'channel')
        image_4D[ch] = [stack[0:z_dim] for stack in image_4D[ch]]
        image_4D[ch] = np.array(image_4D[ch])
        # for tim in tqdm(range(len(image_4D[ch])), desc = 'setting img_limits'):
            # if image_4D[ch][tim].min()!= 0 or image_4D[ch][tim].dtype != ddtype:
            #     image_4D[ch][tim] = img_limits(image_4D[ch][tim], limit=0, ddtype=ddtype)    
    # if save == True:
    #     if save_path[-1] != '/':
    #         save_path += '/'
    #     if save_file == '':
    #         name1 = os.path.basename(files_list[0])
    #         name2 = os.path.basename(files_list[1])
    #         for s in name1:
    #             if s in name2:
    #                 save_file += s
    #             else:
    #                 break
    #     for ch, img in image_4D.items():
    #         save_name = save_path+'4D_'+ch+'_'+save_file
    #         if os.path.splitext(save_name)[-1] not in ['.tif','.tiff']:
    #             save_name += '.tif'
    #         save_image(save_name, img, xy_pixel=xy_pixel, z_pixel=z_pixel)
    print('files_to_4D runtime', timer()-start_time)
    return image_4D

In [6]:
def check_similarity(ref, image):
    try:
        for i in [1]:
            image = image.numpy()
            ref = ref.numpy()
    except:
        pass
    check = sum(metrics.pairwise.cosine_similarity(image.ravel().reshape(1,-1), 
                           ref.ravel().reshape(1,-1)))[0]
    # print('check_similarity of image to ref is', check)
    return check

In [7]:
def similarity_4D(image_4D, save=True, save_path='', save_file=''):
    start_time = timer()
    similairties = {1:1}
    for t in tqdm(np.arange(len(image_4D[1:])), desc='cosine_sim for timepoint'):
        img_t = image_4D[t]
        similairties[t+2] = check_similarity(img_t, image_4D[t+1])
    if save == True:
        if save_file == '':
            save_file = "phase_similarity_check.csv"
        checks_file = save_path+save_file
        if '.csv' not in checks_file:
            checks_file +='.csv'
        with open(checks_file, 'w', newline='') as csvfile:
            fieldnames = ['timepoint', 'cosine_similarity']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            for timepoint, check in similairties.items():
                writer.writerow({'timepoint' : timepoint, 'cosine_similarity' : check})
        csvfile.close()
        print('finished measuring similarity check', timer()-start_time)
    return similairties

In [8]:
def antspy_drift_corr(img_4D_r, img_4D_g, save_path, save_name, ch_names, group_by, name_shifts, ref_t=0, drift_corr='Rigid'):
    """
    This function takes the folder containing all the files to each channel as an input and drift corrects them.
    It uses the files from ch1 to do the drift correction and applies the same correction to ch2.
    By default antspy uses the SyN drift correction algorithm.
    
    path_to_data_ch1: Path to files for ch1 
    path_to_data_ch2: Path to files for ch2
    savepath: Path to folder where the final files should be saved in
    name_ch1: Name for the final file for ch1
    name_ch2: Name for the final file for ch2
    name_shifts: Name for the file containing the shifts for each volume
    """
            
    last = ants.from_numpy(np.float32(filters.median(img_4D_r[-1])))

    shifts = []

    scope = np.arange(ref_t-1:-1,-1)
    scope = np.concatenate((scope, np.arange(ref_t-1,len(img_4D_r)-1)))

    for i in tqdm(scope):
        
        moving_ch1 = ants.from_numpy(np.float32(img_4D_g[i]))
        moving_ch2 = ants.from_numpy(np.float32(img_4D_r[i]))
        
        shift = ants.registration(fixed=last, moving=moving_ch2, type_of_transform=drift_corr)
        
        vol_shifted_ch1 = ants.apply_transforms(fixed=last, moving=moving_ch1, transformlist=shift['fwdtransforms'])
        vol_shifted_ch2 = last = shift['warpedmovout']
                
        io.imsave(save_path+ch_names[0]+save_name+str(f"{i+1:03d}")+'.tif', vol_shifted_ch1.numpy())
        io.imsave(save_path+ch_names[1]+save_name+str(f"{i+1:03d}")+'.tif', vol_shifted_ch2.numpy())

        shifts.append(shift['fwdtransforms'])
                
        del vol_shifted_ch1, vol_shifted_ch2, shift

    gfp_list = get_file_names(path=save_path, group_by=ch_names[0]+save_name, order=False, 
                                nested_files=False, criteria='tif')
    gfp_4D = files_to_4D(files_list=gfp_list, ch_names=[ch_names[0]])
    gfp_name = drift_corr+'_4D_'+ch_names[0]+'_'+group_by+'.tif'
    tif.imwrite(gfp_name, gfp_4D, imagej=True, resolution=(1./0.076, 1./0.076),
                metadata={'spacing': 0.4, 'unit': 'um', 'finterval': 1/10,'axes': 'TZYX'})
    for file in gfp_list:
        os.remove(file)

    red_list = get_file_names(path=save_path, group_by=ch_names[1]+save_name, order=False, 
                                nested_files=False, criteria='tif')
    red_4D = files_to_4D(files_list=red_list, ch_names=[ch_names[1]])
    red_name = drift_corr+'_4D_'+ch_names[1]+'_'+group_by+'.tif'
    tif.imwrite(red_name, red_4D, imagej=True, resolution=(1./0.076, 1./0.076),
                metadata={'spacing': 0.4, 'unit': 'um', 'finterval': 1/10,'axes': 'TZYX'})
    for file in red_list:
        os.remove(file)

    sim_checks = [1]
    for i in tqdm(np.arange(len(gfp_4D)-1), desc='similarity_check'):
        sim_checks.append(check_similarity(gfp_4D[i], gfp_4D[i+1]))
    check_file = save_path+method+'ANTsCheck_'+save_name+".csv"
    check_file = check_file.replace('.tif','')
    with open(check_file, 'w', newline='') as csvfile:
        fieldnames = ['timepoint', 'sim_check']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for timepoint, check in enumerate(sim_checks):
            writer.writerow({'timepoint' : timepoint+1, 'sim_check' : check})
    csvfile.close() 

    np.savetxt(X=shifts, fname=save_path+name_shifts, delimiter=', ', fmt='% s')
    del red_4D, gfp_4D
    return

Running the pipeline

In [ ]:
files_list = get_file_names(path=path_to_data, group_by=group_by, order=True, nested_files=False, criteria='tif')
image_4D = files_to_4D(files_list=files_list, ch_names=ch_names)

In [ ]:
for method in ants_method:
    start_time = timer()
    name_ch1 = method+'_'+ch_names[0]+'_'
    name_ch2 = method+'_'+ch_names[1]+'_'
    name_shifts = method+csv_name
    antspy_drift_corr(img_4D_r=image_4D[ch_names[0]],
                      img_4D_g=image_4D[ch_names[0]], 
                      save_path=save_path, 
                      save_name=save_name,
                      name_ch1=name_ch1,
                      name_ch2=name_ch2,
                      name_shifts=name_shifts,
                      drift_corr=method)

    print(method, ' is done, and it took', timer()-start_time)   

print('script is done')